In [1]:
#System and standard inputs
import sys
import importlib
import timeit
import time
import collections

#Adding correct paths to custom modules
sys.path.append('/Users/sasrey/AlphaTrade/gym_exchange')
sys.path.append('/Users/sasrey/AlphaTrade')
sys.path.append('/Users/sasrey/AlphaTrade/gymnax_exchange')

In [2]:
#Relevant Jax Imports
import jax.numpy as jnp
from jax import lax
import jax

In [365]:
#Direct Import of the Orderbook functions 
import jaxob.JaxOrderbook as job 
job=importlib.reload(job)
#OrderBook class import as written by Kang
import orderbook.orderbook
importlib.reload(orderbook.orderbook)
importlib.reload(orderbook.ordertree)
from orderbook.orderbook import OrderBook as cpuOrderBook
#Import (and reload for debugging) of the OrderBook class with jax backend (SF) should behave in the same way.
import jorderbook
importlib.reload(jorderbook)
from jorderbook import OrderBook as JaxOrderBook

#Import Data Mgt and Encoding/Decoding tools
#(The aim of these is to adjust for the case where no message is received but data ends up in the L2 data)
import gym_exchange.data_orderbook_adapter.data_pipeline as dpl
from gym_exchange.data_orderbook_adapter.decoder import Decoder
from gym_exchange.data_orderbook_adapter.encoder import Encoder

#Attempt at using the data pipeline tools and the encoder/decoder, doesn't really work well. 

dplOBJ=dpl.DataPipeline()
decodeddata=Decoder(10,40,dplOBJ.historical_data,dplOBJ.data_loader)
#Decoder Functionality: Takes the message data (dataloader) and the orderbook data (historical data).
#Creates and maintains an orderbook as implemented in the CPU version.
#Creates and maintains a "DataAdjuster" object, which considers the historical data (orderbook) to be the "true" state
#Inits the orderbook with the first state in the orderbook data (hist) gives imaginary list of IDs to the orders created:
#each entry in the orderbook is considered to be from a single order. 
output=decodeddata.process()
#Process function for Decoder: goes through N (horizon) messages to ascertain whether there are adidtional messages to consider. 

enc=Encoder(decodeddata)
#enc()

In [354]:
import pandas as pd
df_msg = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_message_50.csv", header=None)
df_ob = pd.read_csv("/Users/sasrey/AlphaTrade/data/TSLA_2015-01-02_34200000_57600000_orderbook_50.csv", header=None)
orderbookLevels=50
df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})

In [355]:
data=jnp.array(df_ob.iloc[0,:]).reshape(int(orderbookLevels*2),2) ##Careful - converts to int32 in jnp array, meaning the filler values get truncated.
newarr=jnp.zeros((int(orderbookLevels*2),8))
initOB=newarr.at[:,3].set(data[:,0]).at[:,2].set(data[:,1]).at[:,0].set(1).at[0:orderbookLevels*4:2,1].set(-1).at[1:orderbookLevels*4:2,1].set(1).at[:,4].set(0).at[:,5].set(job.INITID).at[:,6].set(34200).at[:,7].set(0).astype('int32')


df_orders=df_msg.rename(columns={1:'Type',5:'Side',3:'Quantity',4:'Price',2:'OrderID',0:'Time'})
df_orders['TradeID']=0
col_order=['Type','Side','Quantity','Price','TradeID','OrderID','Time']
df_orders=df_orders[col_order]
df_orders_simple=df_orders[(df_orders['Type']!=6)&(df_orders['Type']!=7)&(df_orders['Type']!=5)]


In [356]:
test=df_orders["Time"].astype('string').str.split('.',expand=True)
df_orders[["TimeWhole","TimeDec"]]=test.astype('int32')#.rename(['TimeWhole','TimeDec'])
df_orders=df_orders.drop("Time",axis=1)
ordersJNP=jnp.array(df_orders)
print(ordersJNP.shape)

(290737, 8)


In [374]:
#horizon=13519
horizon=16643
#horizon=87
orderbook_1=JaxOrderBook(price_levels=150,orderQueueLen=100)
trades_init=orderbook_1.process_orders_array(initOB)
t1=time.time()
trades_msgs=orderbook_1.process_orders_array(ordersJNP[0:horizon]).block_until_ready()
t2=time.time()
#trades_msgs=orderbook_1.process_orders_array(ordersJNP[horizon-1:horizon])
print(t2-t1)

7.351049900054932


In [375]:
MINPRICE=-1.410065407e9
myOB=orderbook_1.get_L2_state()
L2OB=jnp.array(df_ob.iloc[horizon-1])
L2OB=L2OB.at[L2OB==MINPRICE].set(-1)


'''
print(ordersJNP[horizon])
print('##################   ORDERBOOK DELTA   ##########################')
print((myOB.reshape(100,4)[0:50,:]-L2OB.reshape(50,4)))#L2OB#-myOB
print('############################################')
print(L2OB.reshape(50,4))
print('############################################')
print(myOB.reshape(100,4)[0:60,:])
'''




"\nprint(ordersJNP[horizon])\nprint('##################   ORDERBOOK DELTA   ##########################')\nprint((myOB.reshape(100,4)[0:50,:]-L2OB.reshape(50,4)))#L2OB#-myOB\nprint('############################################')\nprint(L2OB.reshape(50,4))\nprint('############################################')\nprint(myOB.reshape(100,4)[0:60,:])\n"

## Load the original orderbook implementation on the CPU

In [376]:
def type_side_converter(type : int, side : int):
    side_ret = 'bid' if side ==1 else 'ask'
    if type==1:
        return 'limit',side_ret
    elif type==2:
        return 'cancel',side_ret
    elif type==3:
        return 'delete',side_ret
    elif type==4:
        side_ret = 'ask' if side ==1 else 'bid'
        return 'limit',side_ret
    else:
        return 'skip',side_ret

In [377]:
orderbook_2=cpuOrderBook()
limit_orders = []
order_id_list = [orderbook.orderbook.INITID + i for i in range(100)]
for i in range(50):
    trade_id = 10086
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'ask', 
          'quantity' : df_ob.iloc[0,4* i + 1], 
          'price' : df_ob.iloc[0,4*i],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
for i in range(50):
    trade_id = 10087
    # timestamp = datetime(34200.000000001)
    timestamp = str(34200.000000001)
    item = {'type' : 'limit', 
        'side' : 'bid', 
          'quantity' : df_ob.iloc[0,4* i + 3], 
          'price' : df_ob.iloc[0,4*i+2],
          'trade_id' : trade_id,
          'order_id' : order_id_list[i],#orderbook.orderbook.INITID,
          "timestamp": timestamp}
    if item['quantity']>0:
        limit_orders.append(item)
    
# Add init orders to order book
for order in limit_orders:
    # breakpoint()
    trades, order_id = orderbook_2.process_order(order, True, False)  

print(orderbook_2)

====================================== Asks ======================================
Quantity    30  |  Price  2900000  |  Order_ID   90000049  |  Time 34200.000000001
Quantity    16  |  Price  2887300  |  Order_ID   90000048  |  Time 34200.000000001
Quantity    15  |  Price  2850000  |  Order_ID   90000047  |  Time 34200.000000001
Quantity    10  |  Price  2796200  |  Order_ID   90000046  |  Time 34200.000000001
Quantity    50  |  Price  2780000  |  Order_ID   90000045  |  Time 34200.000000001
Quantity    20  |  Price  2750000  |  Order_ID   90000044  |  Time 34200.000000001
Quantity   100  |  Price  2675100  |  Order_ID   90000043  |  Time 34200.000000001
Quantity    10  |  Price  2675000  |  Order_ID   90000042  |  Time 34200.000000001
Quantity   100  |  Price  2674900  |  Order_ID   90000041  |  Time 34200.000000001
Quantity   100  |  Price  2674600  |  Order_ID   90000040  |  Time 34200.000000001
Quantity    76  |  Price  2656200  |  Order_ID   90000039  |  Time 34200.000000001
Quan

In [378]:
##Taken largely from the Example.py file in the orderbook repository for the cpu
t1=time.time()
for index in range(horizon):
    l1 = df_msg.iloc[index,:]
    ttype,side=type_side_converter(l1[1],l1[5])
    quantity = l1[3]
    price = l1[4]
    trade_id = l1[2] # not sure, in the data it is order id
    order_id = trade_id
    timestamp = str(l1[0])
    message = {'type': ttype,'side': side,'quantity': quantity,'price': price,'trade_id': trade_id, "timestamp":timestamp, 'order_id':order_id}
    print(index,message)
    orderbook_2.processOrder(message,True,False)

t2=time.time()
        
print("The Final order is:")
print(message)

0 {'type': 'skip', 'side': 'bid', 'quantity': 13, 'price': 2228300, 'trade_id': 0, 'timestamp': '34200.00678936', 'order_id': 0}
1 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 1870200, 'trade_id': 7381730, 'timestamp': '34200.030569044', 'order_id': 7381730}
2 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2585700, 'trade_id': 7381891, 'timestamp': '34200.030920932', 'order_id': 7381891}
3 {'type': 'skip', 'side': 'ask', 'quantity': 17011, 'price': 2228700, 'trade_id': -1, 'timestamp': '34200.034289918', 'order_id': -1}
4 {'type': 'limit', 'side': 'bid', 'quantity': 11, 'price': 2227500, 'trade_id': 5519048, 'timestamp': '34200.034289918', 'order_id': 5519048}
5 {'type': 'limit', 'side': 'ask', 'quantity': 10, 'price': 2230000, 'trade_id': 2142274, 'timestamp': '34200.034289918', 'order_id': 2142274}
6 {'type': 'limit', 'side': 'bid', 'quantity': 200, 'price': 2226000, 'trade_id': 2629509, 'timestamp': '34200.034289918', 'order_id': 2629509}
7 {'type': 'limit'

2819 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2218300, 'trade_id': 17716694, 'timestamp': '34901.439794651', 'order_id': 17716694}
should not be here
Quantity   100  |  Price  2218300  |  Order_ID   17716694  |  Time 34901.346565055
2820 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2217600, 'trade_id': 17717506, 'timestamp': '34901.439882034', 'order_id': 17717506}
2821 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2217600, 'trade_id': 17717506, 'timestamp': '34901.441780353', 'order_id': 17717506}
should not be here
Quantity   100  |  Price  2217600  |  Order_ID   17717506  |  Time 34901.439882034
2822 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2199600, 'trade_id': 17717870, 'timestamp': '34901.468265094', 'order_id': 17717870}
2823 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2222100, 'trade_id': 17720402, 'timestamp': '34901.646983103', 'order_id': 17720402}
2824 {'type': 'delete', 'side': 'ask', 'quantity'

4008 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2214100, 'trade_id': 21666077, 'timestamp': '35179.394583674', 'order_id': 21666077}
should not be here
Quantity   100  |  Price  2214100  |  Order_ID   21666077  |  Time 35178.865829377
4009 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2214000, 'trade_id': 21671900, 'timestamp': '35179.394583674', 'order_id': 21671900}
4010 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2214000, 'trade_id': 21671900, 'timestamp': '35179.394650359', 'order_id': 21671900}
should not be here
Quantity   100  |  Price  2214000  |  Order_ID   21671900  |  Time 35179.394583674
4011 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2213900, 'trade_id': 21671901, 'timestamp': '35179.394650359', 'order_id': 21671901}
4012 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2213900, 'trade_id': 21671901, 'timestamp': '35179.394710476', 'order_id': 21671901}
should not be here
Quantity   100  |  Price  221

4107 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2251800, 'trade_id': 9726535, 'timestamp': '35213.805930546', 'order_id': 9726535}
4108 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2228300, 'trade_id': 22092846, 'timestamp': '35213.805994309', 'order_id': 22092846}
4109 {'type': 'limit', 'side': 'bid', 'quantity': 1, 'price': 2212800, 'trade_id': 21952930, 'timestamp': '35215.144439177', 'order_id': 21952930}
4110 {'type': 'skip', 'side': 'bid', 'quantity': 100, 'price': 2212900, 'trade_id': 0, 'timestamp': '35215.144473996', 'order_id': 0}
4111 {'type': 'skip', 'side': 'bid', 'quantity': 60, 'price': 2212900, 'trade_id': 0, 'timestamp': '35215.144496291', 'order_id': 0}
4112 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2211600, 'trade_id': 22106705, 'timestamp': '35215.144499945', 'order_id': 22106705}
4113 {'type': 'skip', 'side': 'bid', 'quantity': 9, 'price': 2212900, 'trade_id': 0, 'timestamp': '35215.144562949', 'order_id': 0}
4114 {'t

6361 {'type': 'delete', 'side': 'ask', 'quantity': 200, 'price': 2201700, 'trade_id': 24555493, 'timestamp': '35397.541294927', 'order_id': 24555493}
6362 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2201400, 'trade_id': 24566441, 'timestamp': '35397.621639647', 'order_id': 24566441}
6363 {'type': 'delete', 'side': 'ask', 'quantity': 51, 'price': 2204300, 'trade_id': 23953387, 'timestamp': '35397.859828089', 'order_id': 23953387}
6364 {'type': 'limit', 'side': 'ask', 'quantity': 51, 'price': 2202000, 'trade_id': 24568357, 'timestamp': '35397.861344783', 'order_id': 24568357}
6365 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2200200, 'trade_id': 24569556, 'timestamp': '35397.947827919', 'order_id': 24569556}
6366 {'type': 'skip', 'side': 'bid', 'quantity': 94, 'price': 2199500, 'trade_id': 0, 'timestamp': '35398.18035387', 'order_id': 0}
6367 {'type': 'skip', 'side': 'bid', 'quantity': 6, 'price': 2200000, 'trade_id': 0, 'timestamp': '35398.18035387', 'order_

...

10827 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2196300, 'trade_id': 31317662, 'timestamp': '35964.256922439', 'order_id': 31317662}
10828 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2200900, 'trade_id': 31307854, 'timestamp': '35965.330906694', 'order_id': 31307854}
10829 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2196600, 'trade_id': 31324425, 'timestamp': '35965.331143145', 'order_id': 31324425}
10830 {'type': 'skip', 'side': 'bid', 'quantity': 100, 'price': 2190900, 'trade_id': 0, 'timestamp': '35966.970715375', 'order_id': 0}
10831 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2183600, 'trade_id': 31317660, 'timestamp': '35966.971018292', 'order_id': 31317660}
should not be here
Quantity   100  |  Price  2183600  |  Order_ID   31317660  |  Time 35964.256897356
10832 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2196300, 'trade_id': 31317662, 'timestamp': '35966.971033777', 'order_id': 31317662}
10833 {'ty

12232 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2182400, 'trade_id': 33712337, 'timestamp': '36114.703717148', 'order_id': 33712337}
12233 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2182600, 'trade_id': 33716297, 'timestamp': '36114.703799509', 'order_id': 33716297}
12234 {'type': 'limit', 'side': 'ask', 'quantity': 300, 'price': 2183600, 'trade_id': 33716480, 'timestamp': '36114.71448321', 'order_id': 33716480}
12235 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2180800, 'trade_id': 33717773, 'timestamp': '36114.791131268', 'order_id': 33717773}
12236 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2181100, 'trade_id': 33717781, 'timestamp': '36114.791735507', 'order_id': 33717781}
12237 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2182600, 'trade_id': 33716297, 'timestamp': '36114.794005734', 'order_id': 33716297}
12238 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2182700, 'trade_id': 33717821, 'ti

13343 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2178400, 'trade_id': 34814768, 'timestamp': '36162.344008871', 'order_id': 34814768}
13344 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2173400, 'trade_id': 34814791, 'timestamp': '36162.344023029', 'order_id': 34814791}
13345 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2177700, 'trade_id': 34814733, 'timestamp': '36162.371780918', 'order_id': 34814733}
13346 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2177900, 'trade_id': 34814781, 'timestamp': '36162.373569878', 'order_id': 34814781}
13347 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2173400, 'trade_id': 34814791, 'timestamp': '36162.373681591', 'order_id': 34814791}
should not be here
Quantity   100  |  Price  2173400  |  Order_ID   34814791  |  Time 36162.344023029
13348 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2176500, 'trade_id': 34815236, 'timestamp': '36162.385348981', 'order_id': 3481

14355 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2169000, 'trade_id': 35723144, 'timestamp': '36220.131023733', 'order_id': 35723144}
14356 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2168400, 'trade_id': 35723138, 'timestamp': '36220.131078168', 'order_id': 35723138}
should not be here
Quantity   100  |  Price  2168400  |  Order_ID   35723138  |  Time 36220.130868149
14357 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2168300, 'trade_id': 35721578, 'timestamp': '36220.132831912', 'order_id': 35721578}
should not be here
Quantity   100  |  Price  2168300  |  Order_ID   35721578  |  Time 36220.052766694
14358 {'type': 'limit', 'side': 'bid', 'quantity': 100, 'price': 2168600, 'trade_id': 35723193, 'timestamp': '36220.132904657', 'order_id': 35723193}
14359 {'type': 'delete', 'side': 'bid', 'quantity': 100, 'price': 2169000, 'trade_id': 35723144, 'timestamp': '36220.161026485', 'order_id': 35723144}
should not be here
Quantity   100  |  Price

15741 {'type': 'delete', 'side': 'ask', 'quantity': 100, 'price': 2172500, 'trade_id': 37721018, 'timestamp': '36328.797185178', 'order_id': 37721018}
15742 {'type': 'limit', 'side': 'ask', 'quantity': 100, 'price': 2172200, 'trade_id': 37786479, 'timestamp': '36328.798146661', 'order_id': 37786479}
15743 {'type': 'delete', 'side': 'ask', 'quantity': 1, 'price': 2172400, 'trade_id': 37689919, 'timestamp': '36328.901790426', 'order_id': 37689919}
15744 {'type': 'limit', 'side': 'ask', 'quantity': 1, 'price': 2171700, 'trade_id': 37787916, 'timestamp': '36328.901900219', 'order_id': 37787916}
15745 {'type': 'delete', 'side': 'bid', 'quantity': 49, 'price': 2170000, 'trade_id': 37768722, 'timestamp': '36328.926916227', 'order_id': 37768722}
should not be here
Quantity    49  |  Price  2170000  |  Order_ID   37768722  |  Time 36327.686176079
15746 {'type': 'skip', 'side': 'bid', 'quantity': 49, 'price': 2170950, 'trade_id': 0, 'timestamp': '36328.927674694', 'order_id': 0}
15747 {'type': '

In [385]:
ordersJNP[horizon] #25590397

Array([        3,         1,        20,   2160000,         0,  26671093,
           36395, 537433168], dtype=int32)

In [380]:
t2-t1

1.0526752471923828

In [381]:
from itertools import zip_longest
cpuOB=jnp.array(list(zip_longest(*orderbook_2.get_L2_state(), fillvalue=-1)))

print('#'*15+" Difference between the JAX (GPU) and Python (CPU) orderbook implementations"+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print((cpuOB[:50,:]-myOB.reshape(150,4)[0:50,:]))
print('#'*15+" Python (CPU) orderbook "+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print(cpuOB[:50,:])
print('#'*15+" JAX (GPU) orderbook "+'#'*15)
print('#'*15+" Horizon: "+str(horizon)+' Param2: '+'#'*15)
print(myOB.reshape(150,4)[0:50,:])

############### Difference between the JAX (GPU) and Python (CPU) orderbook implementations###############
############### Horizon: 16643 Param2: ###############
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
############### Python (CPU) orderbook ###############
############### Horizon: 16643 Param2: ###############
[[2161300     100 2160000      29]
 [2161500     200 2159300     100]
 [2161800     200 2158400     102]
 [2161900     100 2157200     100]
 [2162100     100 2156600     100]
 [

In [382]:
print(orderbook_2.bids.get_price_list(2160000))

Quantity    29  |  Price  2160000  |  Order_ID   38916890  |  Time 36391.009541395



In [386]:
print(orderbook_2.bids.order_map[26671093])

Quantity    19  |  Price  2160000  |  Order_ID   26671093  |  Time 35567.228747576


In [383]:
orderbook_1.orderbook_array[1,0,:,:]

Array([[      29,  2160000,        0, 38916890,    36391,  9541395],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       -1,       -1,       -1,       -1],
       [      -1,       -1,       

In [ ]:
testorder = {'type' : 'delete', 
            'side' : 'bid', 
          'quantity' : 300, 
          'price' : 2226400,
          'trade_id' : 0,
          'order_id' : 11,#orderbook.orderbook.INITID,
          "timestamp": '34200.1'}
orderbook_2.processOrder(testorder,from_data=True,verbose=False)
print(orderbook_2)

In [340]:
orderbook.orderbook.INITID

90000000